# 0- Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
    from amplpy import AMPL, tools
except ImportError:
    # Caso ocorra um erro ao importar, instala amplpy e tenta importar novamente
    %pip install amplpy
    from amplpy import AMPL, tools

In [3]:
licenca ="default"

In [4]:

ampl = tools.ampl_notebook(
    modules=["highs", "gcg", "gurobi", "cplex"],  # Módulos a serem instalados
    license_uuid=licenca,  # Licença a ser utilizada
    g=globals()  # Instancia o objeto AMPL e registra os comandos mágicos
)


# 1- Base

Importando a base pré-processada com o limite de produção diário de:
<ul>
    <li>120.000 peças para o “Tipo de buffer” CX e </li>
    <li>100.000 peças para o “Tipo de buffer” EN.</li>
 </ul>

In [5]:
df_process = pd.read_excel('../../01-Data/ordens_pre_process_revisado.xlsx')

df_process

,Unnamed: 0,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Tipo_de_buffer,Item_pai,Cor,Tamanho,Inner,Rota,Capacidade,pai_cor,chave_loja_buffer_onda,Volume_unit
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,CX,1865,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,CX,1865,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,CX,1865,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,CX,2489,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,CX,2988,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,EN,1863,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,EN,1863,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,EN,1863,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,EN,713,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707


In [6]:
df_process = df_process.rename(columns={'Unnamed: 0': "Id"})

In [7]:
df_process = df_process.rename(columns={'Capacidade': "Capacidade_rota"})

In [8]:
df_process.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit'],
      dtype='object')

In [9]:
df_process['tempo_espera_dias'] = (df_process['Data_da_ordem_de_produção'].max() - df_process['Data_da_ordem_de_produção']).dt.days.astype('int16')

In [10]:
# Definir a função para calcular a prioridade
def calcular_prioridade(delta_t):
    diferenca = delta_t_maximo - delta_t
    if diferenca <= 5:
        return delta_t_maximo
    elif diferenca <= 10:
        return delta_t_maximo - 10
    else:
        return delta_t_maximo - 20

In [11]:
delta_t_maximo = df_process['tempo_espera_dias'].max()
df_process['prioridade'] = df_process['tempo_espera_dias'].apply(calcular_prioridade)
df_process

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300,98,98
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500,98,98
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200,98,98
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624,37,78
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500,37,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78


In [12]:
(df_process['Data_da_ordem_de_produção'].max() - df_process['Data_da_ordem_de_produção']).dt.days.astype('int16').head(300)

0      98
1      98
2      98
3      37
4      37
       ..
295    20
296    20
297    20
298    20
299    20
Name: Data_da_ordem_de_produção, Length: 300, dtype: int16

In [13]:
df_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

In [14]:
df_process

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300,98,98
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500,98,98
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200,98,98
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624,37,78
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500,37,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78


## Caixa padrão

In [15]:
df_caixas_tipo = pd.read_excel('../../01-Data/Dados - desafio disciplina RPVMM_edit.xlsx', sheet_name='Tipos de caixa')
df_caixas_tipo

,Tipo,Tipo de buffer,Comprimento,Largura,Altura,Fator de ocupação,Peças max,Peso max (kg)
0,Caixa 1,CX,380,285,285,0.9,170,23
1,Caixa 2,CX,586,436,272,1.2,170,23
2,Caixa 3,CX,728,427,270,0.9,170,23
3,Caixa 1,EN,380,285,285,0.9,512,23
4,Caixa 2,EN,586,436,272,1.2,512,23
5,Caixa 3,EN,728,427,270,0.9,512,23


In [16]:
df_caixas_tipo.columns = df_caixas_tipo.columns.str.replace(' ', '_')

In [17]:
df_caixas_tipo['volume_util_caixa'] = df_caixas_tipo['Comprimento'] * df_caixas_tipo['Largura'] * df_caixas_tipo['Altura'] * df_caixas_tipo['Fator_de_ocupação']

df_caixas_tipo

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
0,Caixa 1,CX,380,285,285,0.9,170,23,27778950.0
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
2,Caixa 3,CX,728,427,270,0.9,170,23,75538008.0
3,Caixa 1,EN,380,285,285,0.9,512,23,27778950.0
4,Caixa 2,EN,586,436,272,1.2,512,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [18]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns=['Tipo','volume_util_caixa'], values=['Peças_max', 'volume_util_caixa']).reset_index()

Tipo_de_buffer  Peças_max                        \
Tipo                                Caixa 1    Caixa 2    Caixa 3   
volume_util_caixa                27778950.0 83393894.4 75538008.0   
0                             CX      170.0      170.0      170.0   
1                             EN      512.0      512.0      512.0   

                  volume_util_caixa                          
Tipo                        Caixa 1     Caixa 2     Caixa 3  
volume_util_caixa        27778950.0  83393894.4  75538008.0  
0                        27778950.0  83393894.4  75538008.0  
1                        27778950.0  83393894.4  75538008.0

In [19]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='volume_util_caixa').add_prefix('volume_util_caixa_').reset_index()

Tipo,Tipo_de_buffer,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,CX,27778950.0,83393894.4,75538008.0
1,EN,27778950.0,83393894.4,75538008.0


In [20]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='Peças_max').add_prefix('Peças_max_').reset_index()

Tipo,Tipo_de_buffer,Peças_max_Caixa 1,Peças_max_Caixa 2,Peças_max_Caixa 3
0,CX,170,170,170
1,EN,512,512,512


In [21]:
df_caixas_tipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tipo               6 non-null      object 
 1   Tipo_de_buffer     6 non-null      object 
 2   Comprimento        6 non-null      int64  
 3   Largura            6 non-null      int64  
 4   Altura             6 non-null      int64  
 5   Fator_de_ocupação  6 non-null      float64
 6   Peças_max          6 non-null      int64  
 7   Peso_max_(kg)      6 non-null      int64  
 8   volume_util_caixa  6 non-null      float64
dtypes: float64(2), int64(5), object(2)
memory usage: 564.0+ bytes


In [22]:
df_caixas_tipo.columns

Index(['Tipo', 'Tipo_de_buffer', 'Comprimento', 'Largura', 'Altura',
       'Fator_de_ocupação', 'Peças_max', 'Peso_max_(kg)', 'volume_util_caixa'],
      dtype='object')

Temos um tipo de caixa padrão para cada tipo de buffer

In [23]:
df_caixas_padrao = df_caixas_tipo.loc[[1, 5]]
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


Levar a capacidade das caixas para cada ordem de produção:

O **número de peças máximo** varia com o buffer. Já o *volume* é igual para ambos os tipos, mudando apenas para o **tipo de caixa**

In [24]:
df_caixa_volume_max= df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='volume_util_caixa').add_prefix('volume_util_caixa_').reset_index()

df_caixa_volume_max

Tipo,Tipo_de_buffer,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,CX,27778950.0,83393894.4,75538008.0
1,EN,27778950.0,83393894.4,75538008.0


In [25]:
df_process_capacitado = df_process.merge(df_caixa_volume_max, on='Tipo_de_buffer', how='inner')

df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3092300,98,98,27778950.0,83393894.4,75538008.0
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3321500,98,98,27778950.0,83393894.4,75538008.0
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3537200,98,98,27778950.0,83393894.4,75538008.0
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,3,inf,2489_674.0,116_CX_6,233624,37,78,27778950.0,83393894.4,75538008.0
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,1,inf,2988_451.0,16_CX_2,775500,37,78,27778950.0,83393894.4,75538008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78,27778950.0,83393894.4,75538008.0
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78,27778950.0,83393894.4,75538008.0
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78,27778950.0,83393894.4,75538008.0
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78,27778950.0,83393894.4,75538008.0


In [26]:
df_process_capacitado.columns = df_process_capacitado.columns.str.replace(' ', '_')

In [27]:
import numpy as np
df_process_capacitado['volume_util_padrao'] = np.where(df_process_capacitado['Tipo_de_buffer'] == "EN", df_process_capacitado['volume_util_caixa_Caixa_3'], df_process_capacitado['volume_util_caixa_Caixa_2'])

In [28]:
df_process_capacitado = df_process_capacitado.merge(df_caixas_padrao[['Tipo_de_buffer', 'Peças_max']], on='Tipo_de_buffer', how='inner')
df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,1865_6.0,26_CX_5,3092300,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,1865_6.0,26_CX_5,3321500,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,1865_6.0,26_CX_5,3537200,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,2489_674.0,116_CX_6,233624,37,78,27778950.0,83393894.4,75538008.0,83393894.4,170
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,2988_451.0,16_CX_2,775500,37,78,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,1863_2.0,48_EN_2,3269280,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,1863_2.0,48_EN_2,4037285,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,1863_2.0,48_EN_2,3231360,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,713_275.0,48_EN_2,4235707,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512


In [29]:
df_process_capacitado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

In [30]:
df_process_capacitado['chave_loja_buffer_onda'].nunique()

1508

In [31]:
df_sorted_pecas = df_process_capacitado.groupby('chave_loja_buffer_onda')['Peças'].sum()
df_sorted_pecas = df_sorted_pecas.reset_index()
df_sorted_pecas

,chave_loja_buffer_onda,Peças
0,100_CX_4,17
1,100_CX_5,2
2,100_CX_6,43
3,100_CX_7,5
4,100_CX_9,28
...,...,...
1503,9_EN_2,225
1504,9_EN_3,1127
1505,9_EN_4,1464
1506,9_EN_5,32


In [32]:
df_sorted_pecas.sort_values(by='Peças', ascending=False).head(20)

,chave_loja_buffer_onda,Peças
68,10_EN_3,3459
676,28_EN_4,3187
519,16_EN_3,2093
702,2_EN_3,2086
57,10_CX_1,1948
731,31_EN_3,1802
754,33_EN_3,1786
822,3_CX_9,1690
1391,8_EN_4,1620
813,3_CX_1,1555


In [33]:
# Selecionar as n lojas maiores de um único buffer
n = 20
lojas_selecionadas_maiores = list((df_sorted_pecas.sort_values(by='Peças', ascending=False).head(n))['chave_loja_buffer_onda'])
lojas_selecionadas_maiores

['10_EN_3',
 '28_EN_4',
 '16_EN_3',
 '2_EN_3',
 '10_CX_1',
 '31_EN_3',
 '33_EN_3',
 '3_CX_9',
 '8_EN_4',
 '3_CX_1',
 '10_EN_4',
 '9_EN_4',
 '10_CX_4',
 '3_EN_3',
 '25_EN_4',
 '31_CX_1',
 '18_EN_4',
 '48_EN_4',
 '8_EN_3',
 '10_EN_1']

In [34]:
df_process_capacitado.query(f"chave_loja_buffer_onda in ({lojas_selecionadas_maiores})")

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
14,14,1429,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,777_1.0,10_CX_1,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
15,15,1499,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,777_1.0,10_CX_1,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
16,16,1499,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,777_95.0,10_CX_1,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
17,17,909,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,777_95.0,10_CX_1,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
18,18,1499,2023-10-24,539354347,250,100,55,0.180,1,CARTON_M,...,777_95.0,10_CX_1,1375000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61584,61584,8067,2023-11-22,878451301,431,234,19,0.215,24,CARTON_M,...,930_283.0,28_EN_4,1916226,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61585,61585,9984,2023-11-22,878451328,424,245,18,0.187,15,CARTON_M,...,930_283.0,28_EN_4,1869840,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61586,61586,1157,2023-11-22,873049031,450,306,23,0.287,2,CARTON_M,...,943_18.0,28_EN_4,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61587,61587,8666,2023-11-22,873049102,450,315,20,0.307,2,CARTON_M,...,943_18.0,28_EN_4,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512


In [35]:
df_process_capacitado.query(f"chave_loja_buffer_onda in ({lojas_selecionadas_maiores})")['Peças'].sum()

35168

In [36]:
df_process_capacitado["Volume_total"] = df_process_capacitado["Volume_unit"] * df_process_capacitado["Peças"]
df_process_capacitado["Peso_total"] = df_process_capacitado["Peso_unit"] * df_process_capacitado["Peças"]

In [37]:
(df_process_capacitado.query('Tipo_de_buffer == "EN"').groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(20)).index

Index(['10_EN_3', '28_EN_4', '16_EN_3', '2_EN_3', '31_EN_3', '33_EN_3',
       '8_EN_4', '10_EN_4', '9_EN_4', '3_EN_3', '25_EN_4', '18_EN_4',
       '48_EN_4', '8_EN_3', '10_EN_1', '23_EN_3', '11_EN_4', '3_EN_4',
       '2_EN_4', '9_EN_3'],
      dtype='object', name='chave_loja_buffer_onda')

In [38]:
(df_process_capacitado.query('Tipo_de_buffer == "CX"').groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(20)).index

Index(['10_CX_1', '3_CX_9', '3_CX_1', '10_CX_4', '31_CX_1', '31_CX_9',
       '40_CX_9', '2_CX_9', '16_CX_6', '2_CX_4', '8_CX_4', '10_CX_9',
       '16_CX_9', '16_CX_1', '16_CX_4', '9_CX_4', '33_CX_9', '23_CX_9',
       '48_CX_4', '2_CX_1'],
      dtype='object', name='chave_loja_buffer_onda')

## Rotas

In [39]:
df_rotas = df_process_capacitado[['Rota', 'Capacidade_rota']].drop_duplicates()
df_rotas

,Rota,Capacidade_rota
0,1,inf
3,3,inf
9,10,2.720000e+10
14,8,4.000000e+10
26,6,2.720000e+10
45,13,2.720000e+10
71,11,2.720000e+10
76,12,1.600000e+10
116,5,2.720000e+10
147,15,4.000000e+10


# 2- Modelagem

## 2.1- HEURÍSTICA - FFD

First Fit Decreasing

In [40]:
import pandas as pd

def first_fit_decreasing(df, bin_restrictions, route_restrictions):
    # Sort items in decreasing order of volume and increasing order of date (FIFO)
    df_sorted = df.copy()
    df_sorted['Inv_Volume_total'] = (1 / df_sorted['Volume_total']).replace(np.inf, 0)
    df_sorted = df_sorted.sort_values(by=['Data_da_ordem_de_produção', 'Inv_Volume_total'], ascending=True)
    
    # Initialize bins list
    bins = []
    leftover_items = pd.DataFrame(columns=df_sorted.columns)  # Initialize leftover items DataFrame
    
    # Flag to track if a split was made
    split_made = True
    items_to_drop = []
    df_sorted = df_sorted.reset_index(drop=True)
    items_to_add = []

    while len(df_sorted) != 0:
        split_made = False
        df_sorted = df_sorted[~df_sorted.index.isin(items_to_drop)]

        # Add needed items to df_sorted
        if len(items_to_add) != 0:
            df_sorted = pd.concat([df_sorted, pd.DataFrame(items_to_add)])
        items_to_add = []
        df_sorted = df_sorted.reset_index(drop=True)
        items_to_drop = []
        
        # Make a copy of the sorted DataFrame
        df_sorted_copy = df_sorted.copy()
        
        # Iterate over items in the copy of the sorted DataFrame
        for index, item in df_sorted_copy.iterrows():
            # Flag to check if the item is placed
            placed = False
            
            # Try to place the item in existing bins
            for bin in bins:
                if (bin['Volume'] + item['Volume_total']) <= bin["vol_limit"] \
                    and (bin['Peso_total'] + item['Peso_total']) <= 23 \
                    and bin['Amt.Items'] + item['Peças'] <= bin["item_limit"] \
                    and item["Loja"] == bin["Loja"] \
                    and item["Rota"] == bin["Rota"] \
                    and item["Tipo_de_buffer"] == bin["Buffer"] \
                    and item["Classe_de_onda"] == bin["Wave"]:
                    
                    bin['Items'].append(index)
                    bin['Volume'] += item['Volume_total']
                    bin['Peso_total'] += item['Peso_total']
                    bin['Amt.Items'] += item['Peças']

                    # Remove the item from the original DataFrame
                    items_to_drop.append(index)
                    placed = True
                    break
            
            # If the item couldn't be placed entirely, reduce it by "inner" means until it fits into a bin
            while not placed and item['Peças'] > item['Inner']:
                # Reduce the item to its inner size
                detached_portion = item.copy()
                detached_portion['Peças'] = item['Inner']
                detached_portion['Volume_total'] = item['Volume_unit'] * item['Inner']
                detached_portion['Peso_total'] = item['Peso_unit'] * item['Inner']
                
                # Update the remaining portion of the item
                item['Peças'] -= item['Inner']
                item['Volume_total'] -= item['Volume_unit'] * item['Inner']
                item['Peso_total'] -= item['Peso_unit'] * item['Inner']
                
                # Concatenate the reduced portion back to the original DataFrame
                items_to_add.append(detached_portion)
                split_made = True
                
                # Try to place the reduced item in existing bins
                for bin in bins:
                    if (bin['Volume'] + item['Volume_total']) <= bin["vol_limit"] \
                        and (bin['Peso_total'] + item['Peso_total']) <= 23 \
                        and bin['Amt.Items'] + item['Peças'] <= bin["item_limit"] \
                        and item["Loja"] == bin["Loja"] \
                        and item["Rota"] == bin["Rota"] \
                        and item["Tipo_de_buffer"] == bin["Buffer"] \
                        and item["Classe_de_onda"] == bin["Wave"]:
                        
                        bin['Items'].append(index)  # Append index of the appended reduced item
                        bin['Volume'] += item['Volume_total']
                        bin['Peso_total'] += item['Peso_total']
                        bin['Amt.Items'] += item['Peças']

                        items_to_drop.append(index) # Remove the reduced item from the original DataFrame
                        placed = True
                        break
            
            # If the item still couldn't be placed, create a new bin
            if not placed:
                # Check if creating a new bin exceeds route capacity
                if len([bin for bin in bins if bin['Rota'] == item["Rota"]]) >= route_restrictions[route_restrictions["Rota"] == item["Rota"]]["Capacidade_rota"].values[0]:
                    leftover_items = leftover_items.append(item)  # Add item to leftover items DataFrame
                else:
                    bins.append({
                        'Items': [index],
                        'Volume': item['Volume_total'],
                        'Peso_total': item['Peso_total'],
                        'Amt.Items': item['Peças'],
                        "Loja": item["Loja"],
                        "Rota": item["Rota"],
                        "Buffer": item["Tipo_de_buffer"],
                        "Wave": item["Classe_de_onda"],
                        "vol_limit": bin_restrictions[bin_restrictions["Tipo_de_buffer"] == item["Tipo_de_buffer"]]["volume_util_caixa"].values[0],
                        "item_limit": bin_restrictions[bin_restrictions["Tipo_de_buffer"] == item["Tipo_de_buffer"]]["Peças_max"].values[0]
                    })
                    items_to_drop.append(index)
                    placed = True
                    break
    
    return bins, leftover_items


In [41]:
def add_item_loja_column(df):
    # Group by 'Loja' and 'Produto' and use cumcount to assign a sequential number starting from 1
    df['item_loja'] = df.groupby(['Loja']).cumcount() + 1
    return df

### 2.1.1- Situação 1

In [42]:
df_ffd_s1 = df_process_capacitado.query("chave_loja_buffer_onda in ('48_EN_4', '8_EN_3', '10_EN_1')")
df_ffd_s1

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
33706,33706,1797,2023-10-18,595814344,505,206,40,0.237,3,CARTON_M,...,4161200,35,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12483600,0.711
33748,33748,5649,2023-10-26,873646759,450,279,22,0.207,2,CARTON_M,...,2762100,27,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5524200,0.414
33749,33749,7066,2023-10-26,873646741,450,293,22,0.207,5,CARTON_M,...,2900700,27,78,27778950.0,83393894.4,75538008.0,75538008.0,512,14503500,1.035
33750,33750,10193,2023-10-26,873646732,450,279,21,0.220,8,CARTON_M,...,2636550,27,78,27778950.0,83393894.4,75538008.0,75538008.0,512,21092400,1.760
33751,33751,10775,2023-10-26,873646716,440,306,23,0.233,2,CARTON_M,...,3096720,27,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6193440,0.466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59452,59452,10640,2023-11-21,841265687,411,300,63,0.615,1,CARTON_M,...,7767900,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,7767900,0.615
59453,59453,10428,2023-11-21,549983673,363,238,16,0.232,1,CARTON_M,...,1382304,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,1382304,0.232
59454,59454,628,2023-11-21,858645213,442,215,18,0.253,3,CARTON_M,...,1710540,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5131620,0.759
59455,59455,4260,2023-11-21,858645221,461,248,21,0.220,6,CARTON_M,...,2400888,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,14405328,1.320


In [43]:
df_ffd_s1['Peças'].sum()

4032

In [44]:
df_ffd_s1['Volume_total'].sum()

17791897922

In [45]:
backlog = df_ffd_s1.copy()
backlog = backlog.reset_index(drop=True)
backlog = add_item_loja_column(backlog)
packed_bins, leftovers = first_fit_decreasing(df_ffd_s1, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")


Packed bins (new method):
Bin 1: Amt. Items 12 (Total Volume: 74873786 cubic mm | Total Weight: 3.1370kg)
Bin 2: Amt. Items 25 (Total Volume: 74936890 cubic mm | Total Weight: 5.1740kg)
Bin 3: Amt. Items 23 (Total Volume: 75521712 cubic mm | Total Weight: 7.6300kg)
Bin 4: Amt. Items 23 (Total Volume: 74822084 cubic mm | Total Weight: 6.5500kg)
Bin 5: Amt. Items 33 (Total Volume: 75227936 cubic mm | Total Weight: 5.4000kg)
Bin 6: Amt. Items 21 (Total Volume: 74732851 cubic mm | Total Weight: 4.7800kg)
Bin 7: Amt. Items 16 (Total Volume: 75497416 cubic mm | Total Weight: 5.0700kg)
Bin 8: Amt. Items 14 (Total Volume: 75197193 cubic mm | Total Weight: 5.4250kg)
Bin 9: Amt. Items 15 (Total Volume: 75404806 cubic mm | Total Weight: 5.8090kg)
Bin 10: Amt. Items 11 (Total Volume: 74240928 cubic mm | Total Weight: 3.7610kg)
Bin 11: Amt. Items 21 (Total Volume: 75455668 cubic mm | Total Weight: 4.7010kg)
Bin 12: Amt. Items 18 (Total Volume: 75522985 cubic mm | Total Weight: 3.5290kg)
Bin 13: Amt

### 2.1.2- Situação 2

In [46]:
df_ffd_s2 = df_process_capacitado.query("chave_loja_buffer_onda in ('25_EN_4', '31_CX_1', '18_EN_4','3_EN_3')")

df_ffd_s2

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
123,123,871,2023-10-24,534996762,364,167,90,0.090,2,CARTON_M,...,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,10941840,0.180
124,124,1321,2023-10-24,534996754,359,146,76,0.080,4,CARTON_M,...,3983464,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,15933856,0.320
125,125,871,2023-10-24,552747022,250,100,9,0.108,1,CARTON_M,...,225000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,225000,0.108
126,126,1321,2023-10-24,539354355,250,100,70,0.200,1,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1750000,0.200
127,127,871,2023-10-24,539354339,250,100,58,0.175,1,CARTON_M,...,1450000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1450000,0.175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61445,61445,5930,2023-11-22,878451336,427,236,19,0.173,2,CARTON_M,...,1914668,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,3829336,0.346
61446,61446,8067,2023-11-22,878451301,431,234,19,0.215,7,CARTON_M,...,1916226,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,13413582,1.505
61447,61447,8022,2023-11-22,878451310,429,240,19,0.198,9,CARTON_M,...,1956240,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,17606160,1.782
61448,61448,9984,2023-11-22,878451328,424,245,18,0.187,4,CARTON_M,...,1869840,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,7479360,0.748


In [47]:
df_ffd_s2['Peças'].sum()

5496

In [48]:
df_ffd_s2['Volume_total'].sum()

23263872349

In [49]:
backlog = df_ffd_s2.copy()
backlog = backlog.reset_index(drop=True)
backlog = add_item_loja_column(backlog)
packed_bins, leftovers = first_fit_decreasing(df_ffd_s2, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")


Packed bins (new method):
Bin 1: Amt. Items 42 (Total Volume: 83340714 cubic mm | Total Weight: 4.6510kg)
Bin 2: Amt. Items 23 (Total Volume: 75084452 cubic mm | Total Weight: 7.0840kg)
Bin 3: Amt. Items 20 (Total Volume: 75527800 cubic mm | Total Weight: 5.7680kg)
Bin 4: Amt. Items 23 (Total Volume: 74688420 cubic mm | Total Weight: 6.2580kg)
Bin 5: Amt. Items 20 (Total Volume: 75521088 cubic mm | Total Weight: 5.3720kg)
Bin 6: Amt. Items 20 (Total Volume: 74183557 cubic mm | Total Weight: 5.4950kg)
Bin 7: Amt. Items 17 (Total Volume: 75257553 cubic mm | Total Weight: 4.5730kg)
Bin 8: Amt. Items 16 (Total Volume: 75357594 cubic mm | Total Weight: 5.2100kg)
Bin 9: Amt. Items 22 (Total Volume: 75414446 cubic mm | Total Weight: 5.4550kg)
Bin 10: Amt. Items 11 (Total Volume: 75514248 cubic mm | Total Weight: 4.0450kg)
Bin 11: Amt. Items 19 (Total Volume: 75470603 cubic mm | Total Weight: 4.3020kg)
Bin 12: Amt. Items 14 (Total Volume: 75448341 cubic mm | Total Weight: 4.1590kg)
Bin 13: Amt

### 2.1.3- Situação 3

In [50]:
df_ffd_s3 = df_process_capacitado.query("chave_loja_buffer_onda in ('3_CX_1', '10_EN_4', '9_EN_4', '10_CX_4','8_EN_4')")
df_ffd_s3

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
116,116,1470,2023-10-24,534996754,359,146,76,0.080,1,CARTON_M,...,3983464,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3983464,0.080
117,117,1470,2023-10-24,552747022,250,100,9,0.108,1,CARTON_M,...,225000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,225000,0.108
118,118,1470,2023-10-24,534996800,365,230,45,0.220,4,CARTON_M,...,3777750,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,15111000,0.880
119,119,1470,2023-10-24,539354355,250,100,70,0.200,3,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5250000,0.600
120,120,929,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3500000,0.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59786,59786,1204,2023-11-21,879747286,321,146,45,0.240,5,CARTON_M,...,2108970,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,10544850,1.200
59787,59787,4234,2023-11-21,879747315,309,156,65,0.203,2,CARTON_M,...,3133260,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6266520,0.406
59788,59788,8697,2023-11-21,879747294,314,134,41,0.230,7,CARTON_M,...,1725116,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12075812,1.610
59789,59789,9768,2023-11-21,879747307,320,147,42,0.217,3,CARTON_M,...,1975680,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5927040,0.651


In [51]:
df_ffd_s3['Peças'].sum()

7589

In [52]:
df_ffd_s3['Volume_total'].sum()

30081014981

In [53]:
backlog = df_ffd_s3.copy()
backlog = backlog.reset_index(drop=True)
backlog = add_item_loja_column(backlog)
packed_bins, leftovers = first_fit_decreasing(df_ffd_s3, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")


Packed bins (new method):
Bin 1: Amt. Items 44 (Total Volume: 83368971 cubic mm | Total Weight: 7.0660kg)
Bin 2: Amt. Items 20 (Total Volume: 75458526 cubic mm | Total Weight: 7.3600kg)
Bin 3: Amt. Items 23 (Total Volume: 75362040 cubic mm | Total Weight: 7.7170kg)
Bin 4: Amt. Items 25 (Total Volume: 75520446 cubic mm | Total Weight: 7.7330kg)
Bin 5: Amt. Items 22 (Total Volume: 75051750 cubic mm | Total Weight: 7.0930kg)
Bin 6: Amt. Items 19 (Total Volume: 75186105 cubic mm | Total Weight: 4.8070kg)
Bin 7: Amt. Items 22 (Total Volume: 75289616 cubic mm | Total Weight: 4.7270kg)
Bin 8: Amt. Items 14 (Total Volume: 75219294 cubic mm | Total Weight: 5.0800kg)
Bin 9: Amt. Items 19 (Total Volume: 75349756 cubic mm | Total Weight: 6.5560kg)
Bin 10: Amt. Items 13 (Total Volume: 74784992 cubic mm | Total Weight: 4.8930kg)
Bin 11: Amt. Items 13 (Total Volume: 74927770 cubic mm | Total Weight: 3.8650kg)
Bin 12: Amt. Items 28 (Total Volume: 74911604 cubic mm | Total Weight: 7.3710kg)
Bin 13: Amt

### 2.1.4- Situação 4

In [54]:
df_ffd_s4 = df_process_capacitado.query("chave_loja_buffer_onda in ('10_EN_3', '28_EN_4', '16_EN_3')")
df_ffd_s4

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
33800,33800,40,2023-10-30,847254655,443,233,32,0.303,10,CARTON_M,...,3303008,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,33030080,3.030
33801,33801,915,2023-10-30,847254621,445,242,33,0.323,10,CARTON_M,...,3553770,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,35537700,3.230
33802,33802,4051,2023-10-30,847254663,431,225,34,0.295,6,CARTON_M,...,3297150,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,19782900,1.770
33803,33803,8438,2023-10-30,847254639,432,248,32,0.310,12,CARTON_M,...,3428352,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,41140224,3.720
33804,33804,8788,2023-10-30,847254680,445,244,33,0.328,5,CARTON_M,...,3583140,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,17915700,1.640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61584,61584,8067,2023-11-22,878451301,431,234,19,0.215,24,CARTON_M,...,1916226,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,45989424,5.160
61585,61585,9984,2023-11-22,878451328,424,245,18,0.187,15,CARTON_M,...,1869840,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,28047600,2.805
61586,61586,1157,2023-11-22,873049031,450,306,23,0.287,2,CARTON_M,...,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6334200,0.574
61587,61587,8666,2023-11-22,873049102,450,315,20,0.307,2,CARTON_M,...,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5670000,0.614


In [55]:
df_ffd_s4['Peças'].sum()

8739

In [56]:
df_ffd_s4['Volume_total'].sum()

41264636031

In [57]:
backlog = df_ffd_s4.copy()
backlog = backlog.reset_index(drop=True)
backlog = add_item_loja_column(backlog)
packed_bins, leftovers = first_fit_decreasing(df_ffd_s4, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")


Packed bins (new method):
Bin 1: Amt. Items 22 (Total Volume: 75473176 cubic mm | Total Weight: 6.8740kg)
Bin 2: Amt. Items 24 (Total Volume: 74629846 cubic mm | Total Weight: 8.2840kg)
Bin 3: Amt. Items 20 (Total Volume: 73823495 cubic mm | Total Weight: 6.7670kg)
Bin 4: Amt. Items 20 (Total Volume: 74832759 cubic mm | Total Weight: 4.3340kg)
Bin 5: Amt. Items 19 (Total Volume: 74840593 cubic mm | Total Weight: 4.2340kg)
Bin 6: Amt. Items 7 (Total Volume: 75308492 cubic mm | Total Weight: 2.5050kg)
Bin 7: Amt. Items 19 (Total Volume: 75420310 cubic mm | Total Weight: 4.1400kg)
Bin 8: Amt. Items 9 (Total Volume: 74239108 cubic mm | Total Weight: 3.2200kg)
Bin 9: Amt. Items 13 (Total Volume: 74377824 cubic mm | Total Weight: 4.5420kg)
Bin 10: Amt. Items 14 (Total Volume: 74640852 cubic mm | Total Weight: 5.1380kg)
Bin 11: Amt. Items 12 (Total Volume: 75263039 cubic mm | Total Weight: 3.5500kg)
Bin 12: Amt. Items 11 (Total Volume: 75386921 cubic mm | Total Weight: 2.1190kg)
Bin 13: Amt. 

### 2.1.5- Situação 5

In [58]:
df_ffd_s5 = df_process_capacitado.query("chave_loja_buffer_onda in ('2_EN_3',  '10_CX_1', '31_EN_3', '33_EN_3', '3_CX_9')")
df_ffd_s5

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
14,14,1429,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5470920,0.090
15,15,1499,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5470920,0.090
16,16,1499,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3500000,0.400
17,17,909,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3500000,0.400
18,18,1499,2023-10-24,539354347,250,100,55,0.180,1,CARTON_M,...,1375000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1375000,0.180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58412,58412,761,2023-11-21,869448768,702,246,23,0.147,12,CARTON_L,...,3971916,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,47662992,1.764
58413,58413,761,2023-11-21,869448821,295,247,19,0.167,2,CARTON_M,...,1384435,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,2768870,0.334
58414,58414,761,2023-11-21,869448805,407,253,16,0.080,4,CARTON_M,...,1647536,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6590144,0.320
58415,58415,761,2023-11-21,848470202,410,380,35,0.200,1,CARTON_M,...,5453000,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5453000,0.200


In [59]:
df_ffd_s5['Peças'].sum()

9312

In [60]:
df_ffd_s5['Volume_total'].sum()

35353480220

In [61]:
backlog = df_ffd_s5.copy()
backlog = backlog.reset_index(drop=True)
backlog = add_item_loja_column(backlog)
packed_bins, leftovers = first_fit_decreasing(df_ffd_s5, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")


Packed bins (new method):
Bin 1: Amt. Items 49 (Total Volume: 83381956 cubic mm | Total Weight: 6.8580kg)
Bin 2: Amt. Items 14 (Total Volume: 74595451 cubic mm | Total Weight: 2.4480kg)
Bin 3: Amt. Items 7 (Total Volume: 74631704 cubic mm | Total Weight: 2.5970kg)
Bin 4: Amt. Items 16 (Total Volume: 75334857 cubic mm | Total Weight: 3.6620kg)
Bin 5: Amt. Items 66 (Total Volume: 83030748 cubic mm | Total Weight: 10.7070kg)
Bin 6: Amt. Items 9 (Total Volume: 75440864 cubic mm | Total Weight: 1.7200kg)
Bin 7: Amt. Items 11 (Total Volume: 75465144 cubic mm | Total Weight: 3.4150kg)
Bin 8: Amt. Items 12 (Total Volume: 75338845 cubic mm | Total Weight: 3.4730kg)
Bin 9: Amt. Items 13 (Total Volume: 75454000 cubic mm | Total Weight: 3.4930kg)
Bin 10: Amt. Items 41 (Total Volume: 83058870 cubic mm | Total Weight: 8.1940kg)
Bin 11: Amt. Items 14 (Total Volume: 75479026 cubic mm | Total Weight: 3.9150kg)
Bin 12: Amt. Items 26 (Total Volume: 83341294 cubic mm | Total Weight: 4.6500kg)
Bin 13: Amt.

## 2.2- MODELO EXATO

Vamos adicionar uma condição em que o tipo de buffer varia conforme a chave da loja

<p>O peso máximo é mesmo, indenpendente o tipo de buffer. Além disso, nesse momento iremos utilizar o volume padrão para cada buffer</p>

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, para $k$ lojas, cada um com peso $w_{i k}$, um volume $v_{i k}$, um múltiplo $inner_{i k}$ da quantidade de itens a serem selecionados uma demanda $d_{i k}$, para todo $i \in \{1,...,n\}$ e $k \in \{1,...,r\}$, e um número $m$ de mochilas (caixas) diferentes com capacidade volumétrica $cv_{k}$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np_{k}$, então temos:

$$
\begin{aligned}
\text{minimizar} \quad &\sum_{j = 1}^{m} \sum_{k = 1}^{r} y_{j k} \\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i k} \xi_{i j k}*inner_{i k} \leq cp*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\}\\
&\sum_{i = 1}^{n} v_{i k} \xi_{i j k} *inner_{i k} \leq cv_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{i = 1}^{n} \xi_{i j k} *inner_{i k} \leq np_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{j = 1}^{m} \xi_{i j k} *inner_{i k} = d_{i k} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\}\\
&\xi_{i j k} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} , k \in \{1,...,r\} \\
&y_{j k} \in \{0,1\} \quad \forall j \in \{1,...,m\}, k \in \{1,...,r\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j k}$= número de itens tipo $i$ adicionados na caixa $j$, para a loja $k$.


In [62]:
%%ampl_eval
reset;

param n;  # número de tipos de itens
param cp; # capacidade de peso de cada caixa
param r;  # número de lojas (rotas)

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered; # tipos de itens
set K = 1..r ordered; # lojas (rotas)
param w {I,K} >= 0; # peso do item tipo i para a loja k
param v {I,K} >= 0; # volume do item tipo i para a loja k
param inner{I,K} >= 0; # múltiplo da quantidade de itens a serem selecionados
param d {I,K} >= 0; # demanda de itens tipo i para a loja k

param cv {K} >= 0; # capacidade volumétrica de cada loja (rota)
param np {K} >= 0; # capacidade em número de peças para cada loja (rota)

param maxVal := max {i in I, k in K} w[i,k];

param maxVal_peso := max {i in I, k in K} w[i,k];
param pecas_caixa:= floor(cp / maxVal_peso);

param soma := sum{i in I, k in K} d[i, k];
param maxbins := ceil(soma / pecas_caixa);

set J = 1..maxbins ordered;

var e {I,J,K} integer >= 0; # número de itens tipo i na caixa j para a chave loja k
var y {J,K} binary; # 1 se a caixa j é usada para a loja k

minimize FO:  sum {j in J, k in K} y[j,k];

s.t. R1_capacidade_mochila_peso {j in J, k in K}:
   sum {i in I} w[i,k] * e[i,j,k] * inner[i,k] <= cp * y[j,k] suffix block j;

s.t. R2_capacidade_mochila_volume {j in J, k in K}:
   sum {i in I} v[i,k] * e[i,j,k] * inner[i,k] <= cv[k] * y[j,k] suffix block j;
   
s.t. R3_capacidade_mochila_pecas {j in J, k in K}:
   sum {i in I} e[i,j,k] * inner[i,k] <= np[k] * y[j,k] suffix block j;

s.t. R4_quantidade_demanda {i in I, k in K}:
   sum {j in J} e[i,j,k]* inner[i,k] = d[i,k] suffix master 1;

s.t. R5_predecessor_caixas {j in 1..maxbins-1, k in K}:
   y[j,k] >= y[j+1,k];

In [63]:
# %%writefile sit_exemplo.mod


# param n;  # número de tipos de itens
# param cp; # capacidade de peso de cada caixa
# param r;  # número de lojas (rotas)

# suffix master IN, binary;
# suffix block IN, integer;

# set I = 1..n ordered; # tipos de itens
# set K = 1..r ordered; # lojas (rotas)
# param w {I,K} >= 0; # peso do item tipo i para a loja k
# param v {I,K} >= 0; # volume do item tipo i para a loja k
# param inner{I,K} >= 0; # múltiplo da quantidade de itens a serem selecionados
# param d {I,K} >= 0; # demanda de itens tipo i para a loja k

# param cv {K} >= 0; # capacidade volumétrica de cada loja (rota)
# param np {K} >= 0; # capacidade em número de peças para cada loja (rota)

# param maxVal := max {i in I, k in K} w[i,k];

# param maxVal_peso := max {i in I, k in K} w[i,k];
# param pecas_caixa:= floor(cp / maxVal_peso);

# param soma := sum{i in I, k in K} d[i, k];
# param maxbins := ceil(soma / pecas_caixa);

# set J = 1..maxbins ordered;

# var e {I,J,K} integer >= 0; # número de itens tipo i na caixa j para a chave loja k
# var y {J,K} binary; # 1 se a caixa j é usada para a loja k

# minimize FO:  sum {j in J, k in K} y[j,k];

# s.t. R1_capacidade_mochila_peso {j in J, k in K}:
#    sum {i in I} w[i,k] * e[i,j,k] * inner[i,k] <= cp * y[j,k] suffix block j;

# s.t. R2_capacidade_mochila_volume {j in J, k in K}:
#    sum {i in I} v[i,k] * e[i,j,k] * inner[i,k] <= cv[k] * y[j,k] suffix block j;
   
# s.t. R3_capacidade_mochila_pecas {j in J, k in K}:
#    sum {i in I} e[i,j,k] * inner[i,k] <= np[k] * y[j,k] suffix block j;

# s.t. R4_quantidade_demanda {i in I, k in K}:
#    sum {j in J} e[i,j,k]* inner[i,k] = d[i,k] suffix master 1;

### 2.2.1- Situação 1

In [64]:
df_process_situacao1 = df_ffd_s1
df_process_situacao1

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
33706,33706,1797,2023-10-18,595814344,505,206,40,0.237,3,CARTON_M,...,4161200,35,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12483600,0.711
33748,33748,5649,2023-10-26,873646759,450,279,22,0.207,2,CARTON_M,...,2762100,27,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5524200,0.414
33749,33749,7066,2023-10-26,873646741,450,293,22,0.207,5,CARTON_M,...,2900700,27,78,27778950.0,83393894.4,75538008.0,75538008.0,512,14503500,1.035
33750,33750,10193,2023-10-26,873646732,450,279,21,0.220,8,CARTON_M,...,2636550,27,78,27778950.0,83393894.4,75538008.0,75538008.0,512,21092400,1.760
33751,33751,10775,2023-10-26,873646716,440,306,23,0.233,2,CARTON_M,...,3096720,27,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6193440,0.466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59452,59452,10640,2023-11-21,841265687,411,300,63,0.615,1,CARTON_M,...,7767900,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,7767900,0.615
59453,59453,10428,2023-11-21,549983673,363,238,16,0.232,1,CARTON_M,...,1382304,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,1382304,0.232
59454,59454,628,2023-11-21,858645213,442,215,18,0.253,3,CARTON_M,...,1710540,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5131620,0.759
59455,59455,4260,2023-11-21,858645221,461,248,21,0.220,6,CARTON_M,...,2400888,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,14405328,1.320


In [65]:
df_process_situacao1['Peças_max'].unique()

array([512], dtype=int64)

In [66]:
df_process_situacao1.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit', 'tempo_espera_dias',
       'prioridade', 'volume_util_caixa_Caixa_1', 'volume_util_caixa_Caixa_2',
       'volume_util_caixa_Caixa_3', 'volume_util_padrao', 'Peças_max',
       'Volume_total', 'Peso_total'],
      dtype='object')

#### 2.2.1.1- Entrada de dados no modelo

In [67]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,10_EN_1,48_EN_4,8_EN_3
0,1,0.38,0.280,0.237
1,2,0.38,0.260,0.207
2,3,0.38,0.260,0.207
3,4,0.48,0.307,0.220
4,5,0.30,0.287,0.233
...,...,...,...,...
443,444,0.00,0.000,0.615
444,445,0.00,0.000,0.232
445,446,0.00,0.000,0.253
446,447,0.00,0.000,0.220


In [68]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,10_EN_1,48_EN_4,8_EN_3
0,1,3718170.0,3514374.0,4161200.0
1,2,3718170.0,3122875.0,2762100.0
2,3,3718170.0,3122875.0,2900700.0
3,4,4169088.0,3024000.0,2636550.0
4,5,5946556.0,2762540.0,3096720.0
...,...,...,...,...
443,444,0.0,0.0,7767900.0
444,445,0.0,0.0,1382304.0
445,446,0.0,0.0,1710540.0
446,447,0.0,0.0,2400888.0


In [69]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,10_EN_1,48_EN_4,8_EN_3
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
443,444,0.0,0.0,1.0
444,445,0.0,0.0,1.0
445,446,0.0,0.0,1.0
446,447,0.0,0.0,1.0


In [70]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,10_EN_1,48_EN_4,8_EN_3
0,1,4.0,12.0,3.0
1,2,6.0,2.0,2.0
2,3,1.0,1.0,5.0
3,4,1.0,2.0,8.0
4,5,37.0,6.0,2.0
...,...,...,...,...
443,444,0.0,0.0,1.0
444,445,0.0,0.0,1.0
445,446,0.0,0.0,3.0
446,447,0.0,0.0,6.0


Capacidade da caixa como variável

In [71]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao1.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,10_EN_1,512.0
1,48_EN_4,512.0
2,8_EN_3,512.0


In [72]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao1.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,10_EN_1,75538008.0
1,48_EN_4,75538008.0
2,8_EN_3,75538008.0


Características das peças

In [73]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

Características da caixa

In [74]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [75]:
cv['volume_util_padrao'].values

array([75538008., 75538008., 75538008.])

In [76]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

In [77]:
# Função para escrever os parâmetros em um arquivo .dat
# def save_ampl_data(filename, ampl_params):
#     with open(filename, 'w') as f:
#         for param_name, param_value in ampl_params.items():
#             if isinstance(param_value, dict):
#                 f.write(f'param {param_name} :=\n')
#                 for key, value in param_value.items():
#                     if isinstance(key, tuple):
#                         f.write(f'{key} {value}\n')
#                     else:
#                         f.write(f'{key} {value}\n')
#                 f.write(';\n')
#             else:
#                 f.write(f'param {param_name} := {param_value};\n')

# # Dados de exemplo
# ampl_params = {
#     "n": len(pecas),
#     "cp": cp,
#     "r": len(volume.columns) - 1,
#     "w": {(i, k): peso.iloc[i - 1, k] for i in range(1, len(peso) + 1) for k in range(1, len(peso.columns))},
#     "v": {(i, k): volume.iloc[i - 1, k] for i in range(1, len(volume) + 1) for k in range(1, len(volume.columns))},
#     "d": {(i, k): pecas.iloc[i - 1, k] for i in range(1, len(pecas) + 1) for k in range(1, len(pecas.columns))},
#     "inner": {(i, k): inner.iloc[i - 1, k] for i in range(1, len(inner) + 1) for k in range(1, len(inner.columns))},
#     "cv": {k: cv.iloc[k - 1,] for k in range(1, len(cv) + 1)},
#     "np": {k: np_.iloc[k - 1] for k in range(1, len(np_) + 1)}
# }

# # Salvar os dados em um arquivo .dat
# save_ampl_data('dados_ampl.dat', ampl_params)


In [78]:
ampl.getValue('soma')

4032

In [79]:
ampl.getValue('maxbins')

192

In [80]:
ampl.getValue('pecas_caixa')

21

#### 2.2.1.2- Solvers

##### Solver Highs

In [81]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=600"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 600
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+01]
  RHS    [1e+00, 4e+01]
Presolving model
3329 rows, 197952 cols, 792378 nonzeros  0s
3329 rows, 197952 cols, 792378 nonzeros  4s
Objective function is integral with scale 1

Solving MIP model with:
   3329 rows
   197952 cols (51264 binary, 146688 integer, 0 implied int., 0 continuous)
   792378 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     6.6s
         0       0         0   0.00%   235.5357044     inf                  inf        0      0 

In [82]:
%%ampl_eval
display d, w, v, inner;

:       d      w        v     inner    :=
1   1    4   0.38     3718170    1
1   2   12   0.28     3514370    1
1   3    3   0.237    4161200    1
2   1    6   0.38     3718170    1
2   2    2   0.26     3122880    1
2   3    2   0.207    2762100    1
3   1    1   0.38     3718170    1
3   2    1   0.26     3122880    1
3   3    5   0.207    2900700    1
4   1    1   0.48     4169090    1
4   2    2   0.307    3024000    1
4   3    8   0.22     2636550    1
5   1   37   0.3      5946560    1
5   2    6   0.287    2762540    1
5   3    2   0.233    3096720    1
6   1    3   0.32     2968610    1
6   2    2   0.34     2451570    1
6   3    8   0.22     2607000    1
7   1    3   0.32     2968610    1
7   2    1   0.28     3243240    1
7   3    4   0.34    13064200    1
8   1    5   0.32     2968610    1
8   2    1   0.24     3758990    1
8   3    1   0.207    3228150    1
9   1    1   0.29     3930630    1
9   2    3   0.267    3201580    1
9   3    2   0.185    2637180    1
10  1   13   

In [83]:
e_s1_highs = ampl.getVariable('e').getValues().toPandas()
e_s1_highs = e_s1_highs.reset_index()
e_s1_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s1_highs = e_s1_highs.query('valor_multiplo >0')
e_s1_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
22,1,8,2,1.000000e+00
31,1,11,2,3.000000e+00
80,1,27,3,1.000000e+00
109,1,37,2,8.000000e+00
132,1,45,1,2.000000e+00
...,...,...,...,...
256547,446,76,3,2.000000e+00
256556,446,79,3,1.000000e+00
257021,447,42,3,6.000000e+00
257024,447,43,3,1.868948e-16


In [84]:
df_agrupamento_loja1_highs_s1 = e_s1_highs.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_highs_s1 = df_agrupamento_loja1_highs_s1.reset_index()
df_agrupamento_loja1_highs_s1.columns.name = None
df_agrupamento_loja1_highs_s1 # as colunas são as caixas

,item_loja,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,10.0,6.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219,220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
220,221,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
221,222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
vetores_por_loja = {}

# Iterar sobre cada grupo (grupo de cada loja)
for nome_loja, grupo in df_process_situacao1.groupby('chave_loja_buffer_onda'):
    # Extrair os IDs do grupo como um vetor
    vetor_ids = grupo['Id'].tolist()
    # Adicionar o vetor à lista de vetores, usando o nome da loja como chave
    vetores_por_loja[nome_loja] = vetor_ids
    # Atribuir o vetor a uma variável com o nome da loja
    exec(f'vetor_{nome_loja} = vetor_ids')

In [86]:
# Lista dos caracteres finais dos nomes dos vetores
char_final = [col for col in w.columns[:3]]

# Criar o dicionário mapeando os caracteres finais aos vetores
vetores = {char: globals()[f'vetor_{char}'] for char in char_final}

In [87]:
df_agrupamento_loja1_highs_s1['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_highs_s1['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_highs_s1

,item_loja,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33882,10_EN_1
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33883,10_EN_1
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33884,10_EN_1
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33885,10_EN_1
4,5,0.0,0.0,0.0,10.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,33886,10_EN_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57220,10_EN_1
219,220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57221,10_EN_1
220,221,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57222,10_EN_1
221,222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57223,10_EN_1


In [88]:
df_agrupamento_loja2_highs_s1 = e_s1_highs.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_highs_s1 = df_agrupamento_loja2_highs_s1.reset_index()
df_agrupamento_loja2_highs_s1.columns.name = None
df_agrupamento_loja2_highs_s1['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_highs_s1['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_highs_s1

,item_loja,1,2,3,4,5,6,7,8,9,...,93,94,95,96,164,165,187,189,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34260,48_EN_4
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,34261,48_EN_4
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34262,48_EN_4
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34263,48_EN_4
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34264,48_EN_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58702,48_EN_4
353,354,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58703,48_EN_4
354,355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58704,48_EN_4
355,356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58705,48_EN_4


In [89]:
y_s1_highs = ampl.getVariable('y').getValues().toPandas()
y_s1_highs = y_s1_highs.reset_index()
y_s1_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s1_highs = y_s1_highs.query('selecao>0')

df_caixas_por_loja_highs_s1 = y_s1_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s1 = df_caixas_por_loja_highs_s1.reset_index()
df_caixas_por_loja_highs_s1.columns.name = None
df_caixas_por_loja_highs_s1 # as colunas são as lojas

,Caixa,1,2,3
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
91,92,0.0,1.0,0.0
92,93,0.0,1.0,0.0
93,94,0.0,1.0,0.0
94,95,0.0,1.0,0.0


In [90]:
df_caixas_por_loja_highs_s1.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s1

,caixas,10_EN_1,48_EN_4,8_EN_3
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
91,92,0.0,1.0,0.0
92,93,0.0,1.0,0.0
93,94,0.0,1.0,0.0
94,95,0.0,1.0,0.0


##### Caixas

In [91]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s1[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s1[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "10_EN_1": 62.0
número de caixas para a chave_loja_buffer_onda "48_EN_4": 96.0
número de caixas para a chave_loja_buffer_onda "8_EN_3": 83.0

total de caixas: 241.0


### 2.2.2- Situação 2

In [ ]:
df_process_situacao2 = df_ffd_s2

df_process_situacao2

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
123,123,871,2023-10-24,534996762,364,167,90,0.090,2,CARTON_M,...,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,10941840,0.180
124,124,1321,2023-10-24,534996754,359,146,76,0.080,4,CARTON_M,...,3983464,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,15933856,0.320
125,125,871,2023-10-24,552747022,250,100,9,0.108,1,CARTON_M,...,225000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,225000,0.108
126,126,1321,2023-10-24,539354355,250,100,70,0.200,1,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1750000,0.200
127,127,871,2023-10-24,539354339,250,100,58,0.175,1,CARTON_M,...,1450000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1450000,0.175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61445,61445,5930,2023-11-22,878451336,427,236,19,0.173,2,CARTON_M,...,1914668,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,3829336,0.346
61446,61446,8067,2023-11-22,878451301,431,234,19,0.215,7,CARTON_M,...,1916226,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,13413582,1.505
61447,61447,8022,2023-11-22,878451310,429,240,19,0.198,9,CARTON_M,...,1956240,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,17606160,1.782
61448,61448,9984,2023-11-22,878451328,424,245,18,0.187,4,CARTON_M,...,1869840,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,7479360,0.748


#### 2.2.2.1- Entrada de dados no modelo

In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,18_EN_4,25_EN_4,31_CX_1,3_EN_3
0,1,0.260,0.303,0.090,0.180
1,2,0.307,0.323,0.080,0.140
2,3,0.260,0.295,0.108,0.110
3,4,0.300,0.310,0.200,0.437
4,5,0.247,0.328,0.175,0.287
...,...,...,...,...,...
372,373,0.000,0.173,0.000,0.000
373,374,0.000,0.215,0.000,0.000
374,375,0.000,0.198,0.000,0.000
375,376,0.000,0.187,0.000,0.000


In [ ]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,18_EN_4,25_EN_4,31_CX_1,3_EN_3
0,1,3122875.0,3303008.0,5470920.0,2668050.0
1,2,3024000.0,3553770.0,3983464.0,1049364.0
2,3,3576430.0,3297150.0,225000.0,1221480.0
3,4,4237839.0,3428352.0,1750000.0,5118144.0
4,5,3374520.0,3583140.0,1450000.0,4271909.0
...,...,...,...,...,...
372,373,0.0,1914668.0,0.0,0.0
373,374,0.0,1916226.0,0.0,0.0
374,375,0.0,1956240.0,0.0,0.0
375,376,0.0,1869840.0,0.0,0.0


In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,18_EN_4,25_EN_4,31_CX_1,3_EN_3
0,1,1.0,1.0,1.0,1.0
1,2,1.0,1.0,1.0,1.0
2,3,1.0,1.0,1.0,1.0
3,4,1.0,1.0,1.0,1.0
4,5,1.0,1.0,1.0,1.0
...,...,...,...,...,...
372,373,0.0,1.0,0.0,0.0
373,374,0.0,1.0,0.0,0.0
374,375,0.0,1.0,0.0,0.0
375,376,0.0,1.0,0.0,0.0


In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,18_EN_4,25_EN_4,31_CX_1,3_EN_3
0,1,3.0,4.0,2.0,1.0
1,2,3.0,4.0,4.0,1.0
2,3,6.0,3.0,1.0,4.0
3,4,6.0,5.0,1.0,1.0
4,5,2.0,2.0,1.0,2.0
...,...,...,...,...,...
372,373,0.0,2.0,0.0,0.0
373,374,0.0,7.0,0.0,0.0
374,375,0.0,9.0,0.0,0.0
375,376,0.0,4.0,0.0,0.0


Capacidade da caixa como variável

In [ ]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao2.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,18_EN_4,512.0
1,25_EN_4,512.0
2,31_CX_1,170.0
3,3_EN_3,512.0


In [ ]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao2.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,18_EN_4,75538008.0
1,25_EN_4,75538008.0
2,31_CX_1,83393894.4
3,3_EN_3,75538008.0


In [ ]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [ ]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [ ]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

#### 2.2.2.2- Solvers

In [ ]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=600"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 600
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [3e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [3e-16, 3e+01]
  RHS    [1e+00, 5e+01]
Presolving model
Presolve: Infeasible

Solving report
  Status            Infeasible
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  Solution status   -
  Timing            0.11 (total)
                    0.10 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)
Coefficient ranges:
  Matrix [3e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 3e+01]
  RHS    [1e+00, 5e+01]
Presolving model
3604 rows, 225568 cols, 901424 nonzeros  0s
3604 rows, 225568 cols, 901424 nonzeros  3s
Objective function is integral with scale 1

Solving MIP model with:
   3604 rows


In [ ]:
e_s2_highs = ampl.getVariable('e').getValues().toPandas()
e_s2_highs = e_s2_highs.reset_index()
e_s2_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s2_highs = e_s2_highs.query('valor_multiplo >0')
e_s2_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
107,1,27,4,1.0
237,1,60,2,1.0
293,1,74,2,1.0
352,1,89,1,3.0
474,1,119,3,2.0
...,...,...,...,...
318742,376,186,3,1.0
318874,377,7,3,1.0
318993,377,37,2,1.0
319237,377,98,2,1.0


In [ ]:
y_s2_highs = ampl.getVariable('y').getValues().toPandas()
y_s2_highs = y_s2_highs.reset_index()
y_s2_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s2_highs = y_s2_highs.query('selecao>0')

df_caixas_por_loja_highs_s2 = y_s2_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s2 = df_caixas_por_loja_highs_s2.reset_index()
df_caixas_por_loja_highs_s2.columns.name = None
df_caixas_por_loja_highs_s2 # as colunas são as lojas

,Caixa,1,2,3,4
0,1,1.0,0.0,0.0,1.0
1,2,0.0,0.0,0.0,1.0
2,4,0.0,0.0,1.0,0.0
3,5,0.0,1.0,0.0,0.0
4,6,1.0,0.0,0.0,0.0
...,...,...,...,...,...
178,206,0.0,1.0,0.0,0.0
179,207,1.0,0.0,0.0,0.0
180,208,0.0,0.0,0.0,1.0
181,209,1.0,1.0,0.0,1.0


In [ ]:
df_caixas_por_loja_highs_s2.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s2

,caixas,18_EN_4,25_EN_4,31_CX_1,3_EN_3
0,1,1.0,0.0,0.0,1.0
1,2,0.0,0.0,0.0,1.0
2,4,0.0,0.0,1.0,0.0
3,5,0.0,1.0,0.0,0.0
4,6,1.0,0.0,0.0,0.0
...,...,...,...,...,...
178,206,0.0,1.0,0.0,0.0
179,207,1.0,0.0,0.0,0.0
180,208,0.0,0.0,0.0,1.0
181,209,1.0,1.0,0.0,1.0


#### Caixas

In [ ]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s2[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s2[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "18_EN_4": 95.0
número de caixas para a chave_loja_buffer_onda "25_EN_4": 93.0
número de caixas para a chave_loja_buffer_onda "31_CX_1": 36.0
número de caixas para a chave_loja_buffer_onda "3_EN_3": 85.0

total de caixas: 309.0


### 2.2.3- Situação 3

In [ ]:
df_process_situacao3 = df_ffd_s3

df_process_situacao3

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
116,116,1470,2023-10-24,534996754,359,146,76,0.080,1,CARTON_M,...,3983464,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3983464,0.080
117,117,1470,2023-10-24,552747022,250,100,9,0.108,1,CARTON_M,...,225000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,225000,0.108
118,118,1470,2023-10-24,534996800,365,230,45,0.220,4,CARTON_M,...,3777750,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,15111000,0.880
119,119,1470,2023-10-24,539354355,250,100,70,0.200,3,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5250000,0.600
120,120,929,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3500000,0.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59786,59786,1204,2023-11-21,879747286,321,146,45,0.240,5,CARTON_M,...,2108970,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,10544850,1.200
59787,59787,4234,2023-11-21,879747315,309,156,65,0.203,2,CARTON_M,...,3133260,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6266520,0.406
59788,59788,8697,2023-11-21,879747294,314,134,41,0.230,7,CARTON_M,...,1725116,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12075812,1.610
59789,59789,9768,2023-11-21,879747307,320,147,42,0.217,3,CARTON_M,...,1975680,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5927040,0.651


#### 2.2.3.1- Entrada de dados no modelo

In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao3['item_loja'] = df_process_situacao3.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,10_CX_4,10_EN_4,3_CX_1,8_EN_4,9_EN_4
0,1,0.203,0.140,0.080,0.140,0.303
1,2,0.203,0.380,0.108,0.323,0.323
2,3,0.203,0.325,0.220,0.303,0.295
3,4,0.203,0.360,0.200,0.295,0.328
4,5,0.203,0.360,0.200,0.310,0.310
...,...,...,...,...,...,...
505,506,0.000,0.000,0.000,0.300,0.000
506,507,0.000,0.000,0.000,0.267,0.000
507,508,0.000,0.000,0.000,0.280,0.000
508,509,0.000,0.000,0.000,0.313,0.000


In [ ]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,10_CX_4,10_EN_4,3_CX_1,8_EN_4,9_EN_4
0,1,2800000.0,6839430.0,3983464.0,6839430.0,3303008.0
1,2,2000000.0,3448182.0,225000.0,3553770.0,3553770.0
2,3,2800000.0,3448182.0,3777750.0,3303008.0,3297150.0
3,4,2000000.0,2643030.0,1750000.0,3297150.0,3583140.0
4,5,2800000.0,2434600.0,1750000.0,3428352.0,3428352.0
...,...,...,...,...,...,...
505,506,0.0,0.0,0.0,3430336.0,0.0
506,507,0.0,0.0,0.0,3466575.0,0.0
507,508,0.0,0.0,0.0,3342176.0,0.0
508,509,0.0,0.0,0.0,3485328.0,0.0


In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao3['item_loja'] = df_process_situacao3.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,10_CX_4,10_EN_4,3_CX_1,8_EN_4,9_EN_4
0,1,1.0,1.0,1.0,1.0,1.0
1,2,1.0,1.0,1.0,1.0,1.0
2,3,1.0,1.0,1.0,1.0,1.0
3,4,1.0,1.0,1.0,1.0,1.0
4,5,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...
505,506,0.0,0.0,0.0,1.0,0.0
506,507,0.0,0.0,0.0,1.0,0.0
507,508,0.0,0.0,0.0,1.0,0.0
508,509,0.0,0.0,0.0,1.0,0.0


In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao3['item_loja'] = df_process_situacao3.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,10_CX_4,10_EN_4,3_CX_1,8_EN_4,9_EN_4
0,1,4.0,1.0,1.0,1.0,3.0
1,2,6.0,2.0,1.0,2.0,3.0
2,3,5.0,1.0,4.0,2.0,2.0
3,4,11.0,5.0,3.0,1.0,2.0
4,5,15.0,6.0,2.0,2.0,4.0
...,...,...,...,...,...,...
505,506,0.0,0.0,0.0,6.0,0.0
506,507,0.0,0.0,0.0,2.0,0.0
507,508,0.0,0.0,0.0,4.0,0.0
508,509,0.0,0.0,0.0,3.0,0.0


Capacidade da caixa como variável

In [ ]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao3.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,10_CX_4,170.0
1,10_EN_4,512.0
2,3_CX_1,170.0
3,8_EN_4,512.0
4,9_EN_4,512.0


In [ ]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao3.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,10_CX_4,83393894.4
1,10_EN_4,75538008.0
2,3_CX_1,83393894.4
3,8_EN_4,75538008.0
4,9_EN_4,75538008.0


In [ ]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [ ]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [ ]:
len(pecas) 

510

In [ ]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

#### 2.2.3.2- Solvers

In [ ]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=900"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 900
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [3e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 4e+01]
  RHS    [1e+00, 4e+01]
Presolving model
7557 rows, 707560 cols, 2828340 nonzeros  1s
7557 rows, 707560 cols, 2828340 nonzeros  28s
Objective function is integral with scale 1

Solving MIP model with:
   7557 rows
   707560 cols (185820 binary, 521740 integer, 0 implied int., 0 continuous)
   2828340 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0    30.6s
 R       0       0         0   0.00%   387.9816608     591               34.35%        0   

In [ ]:
e_s3_highs = ampl.getVariable('e').getValues().toPandas()
e_s3_highs = e_s3_highs.reset_index()
e_s3_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s3_highs = e_s3_highs.query('valor_multiplo >0')
e_s3_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
50,1,11,1,4
378,1,76,4,1
719,1,144,5,1
809,1,162,5,2
1162,1,233,3,1
...,...,...,...,...
965943,509,149,4,1
966378,509,236,4,1
966768,509,314,4,1
967338,510,48,4,1


In [ ]:
y_s3_highs = ampl.getVariable('y').getValues().toPandas()
y_s3_highs = y_s3_highs.reset_index()
y_s3_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s3_highs = y_s3_highs.query('selecao>0')

df_caixas_por_loja_highs_s3 = y_s3_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s3 = df_caixas_por_loja_highs_s3.reset_index()
df_caixas_por_loja_highs_s3.columns.name = None
df_caixas_por_loja_highs_s3 # as colunas são as lojas

,Caixa,1,2,3,4,5
0,2,0.0,0.0,0.0,1.0,0.0
1,3,0.0,1.0,0.0,0.0,0.0
2,4,1.0,0.0,0.0,1.0,0.0
3,6,0.0,1.0,0.0,0.0,0.0
4,7,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...
265,374,0.0,0.0,0.0,1.0,1.0
266,377,1.0,1.0,0.0,0.0,0.0
267,378,0.0,1.0,0.0,1.0,0.0
268,379,1.0,0.0,0.0,0.0,0.0


In [ ]:
df_caixas_por_loja_highs_s3.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s3

,caixas,10_CX_4,10_EN_4,3_CX_1,8_EN_4,9_EN_4
0,2,0.0,0.0,0.0,1.0,0.0
1,3,0.0,1.0,0.0,0.0,0.0
2,4,1.0,0.0,0.0,1.0,0.0
3,6,0.0,1.0,0.0,0.0,0.0
4,7,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...
265,374,0.0,0.0,0.0,1.0,1.0
266,377,1.0,1.0,0.0,0.0,0.0
267,378,0.0,1.0,0.0,1.0,0.0
268,379,1.0,0.0,0.0,0.0,0.0


#### Caixas

In [ ]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s3[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s3[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "10_CX_4": 59.0
número de caixas para a chave_loja_buffer_onda "10_EN_4": 98.0
número de caixas para a chave_loja_buffer_onda "3_CX_1": 41.0
número de caixas para a chave_loja_buffer_onda "8_EN_4": 101.0
número de caixas para a chave_loja_buffer_onda "9_EN_4": 94.0

total de caixas: 393.0


### 2.2.4- Situação 4

In [ ]:
df_process_situacao4 = df_ffd_s4

df_process_situacao4

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
33800,33800,40,2023-10-30,847254655,443,233,32,0.303,10,CARTON_M,...,3303008,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,33030080,3.030
33801,33801,915,2023-10-30,847254621,445,242,33,0.323,10,CARTON_M,...,3553770,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,35537700,3.230
33802,33802,4051,2023-10-30,847254663,431,225,34,0.295,6,CARTON_M,...,3297150,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,19782900,1.770
33803,33803,8438,2023-10-30,847254639,432,248,32,0.310,12,CARTON_M,...,3428352,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,41140224,3.720
33804,33804,8788,2023-10-30,847254680,445,244,33,0.328,5,CARTON_M,...,3583140,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512,17915700,1.640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61584,61584,8067,2023-11-22,878451301,431,234,19,0.215,24,CARTON_M,...,1916226,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,45989424,5.160
61585,61585,9984,2023-11-22,878451328,424,245,18,0.187,15,CARTON_M,...,1869840,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,28047600,2.805
61586,61586,1157,2023-11-22,873049031,450,306,23,0.287,2,CARTON_M,...,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6334200,0.574
61587,61587,8666,2023-11-22,873049102,450,315,20,0.307,2,CARTON_M,...,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5670000,0.614


#### 2.2.4.1- Entrada de dados no modelo

In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao4['item_loja'] = df_process_situacao4.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao4.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,10_EN_3,16_EN_3,28_EN_4
0,1,0.210,0.220,0.303
1,2,0.140,0.167,0.323
2,3,0.390,0.240,0.295
3,4,0.433,0.213,0.310
4,5,0.310,0.185,0.328
...,...,...,...,...
573,574,0.250,0.000,0.000
574,575,0.250,0.000,0.000
575,576,0.320,0.000,0.000
576,577,0.288,0.000,0.000


In [ ]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao4.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,10_EN_3,16_EN_3,28_EN_4
0,1,4773600.0,3001320.0,3303008.0
1,2,1049364.0,2741220.0,3553770.0
2,3,4320315.0,2864650.0,3297150.0
3,4,6020820.0,2456160.0,3428352.0
4,5,3157578.0,2637180.0,3583140.0
...,...,...,...,...
573,574,1534522.0,0.0,0.0
574,575,1534522.0,0.0,0.0
575,576,2370060.0,0.0,0.0
576,577,2716025.0,0.0,0.0


In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao4['item_loja'] = df_process_situacao4.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao4.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,10_EN_3,16_EN_3,28_EN_4
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
573,574,1.0,0.0,0.0
574,575,1.0,0.0,0.0
575,576,1.0,0.0,0.0
576,577,1.0,0.0,0.0


In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao4['item_loja'] = df_process_situacao4.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao4.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,10_EN_3,16_EN_3,28_EN_4
0,1,1.0,1.0,10.0
1,2,1.0,1.0,10.0
2,3,1.0,4.0,6.0
3,4,1.0,2.0,12.0
4,5,1.0,1.0,5.0
...,...,...,...,...
573,574,5.0,0.0,0.0
574,575,9.0,0.0,0.0
575,576,11.0,0.0,0.0
576,577,6.0,0.0,0.0


Capacidade da caixa como variável

In [ ]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao4.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,10_EN_3,512.0
1,16_EN_3,512.0
2,28_EN_4,512.0


In [ ]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao4.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,10_EN_3,75538008.0
1,16_EN_3,75538008.0
2,28_EN_4,75538008.0


In [ ]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [ ]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [ ]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

#### 2.2.4.2- Solvers

In [ ]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=900"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 900
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [8e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+01]
  RHS    [1e+00, 4e+01]
Presolving model
Presolve: Infeasible

Solving report
  Status            Infeasible
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  Solution status   -
  Timing            0.38 (total)
                    0.35 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)
Coefficient ranges:
  Matrix [8e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 3e+01]
  RHS    [1e+00, 4e+01]
Presolving model
4329 rows, 437763 cols, 1750041 nonzeros  0s
4329 rows, 437763 cols, 1750041 nonzeros  15s
Objective function is integral with scale 1

Solving MIP model with:
   4329 ro

In [ ]:
e_s4_highs = ampl.getVariable('e').getValues().toPandas()
e_s4_highs = e_s4_highs.reset_index()
e_s4_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s4_highs = e_s4_highs.query('valor_multiplo >0')
e_s4_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
367,1,123,2,1.0
717,1,240,1,1.0
791,1,264,3,10.0
1016,2,2,3,9.0
1064,2,18,3,1.0
...,...,...,...,...
581250,575,313,1,8.0
581448,576,42,1,11.0
583287,577,318,1,6.0
584046,578,234,1,10.0


In [ ]:
y_s4_highs = ampl.getVariable('y').getValues().toPandas()
y_s4_highs = y_s4_highs.reset_index()
y_s4_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s4_highs = y_s4_highs.query('selecao>0')

df_caixas_por_loja_highs_s4 = y_s4_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s4 = df_caixas_por_loja_highs_s4.reset_index()
df_caixas_por_loja_highs_s4.columns.name = None
df_caixas_por_loja_highs_s4 # as colunas são as lojas

,Caixa,1,2,3
0,1,1.0,1.0,0.0
1,2,0.0,0.0,1.0
2,4,1.0,1.0,1.0
3,5,1.0,0.0,1.0
4,6,0.0,0.0,1.0
...,...,...,...,...
304,333,1.0,1.0,1.0
305,334,1.0,0.0,0.0
306,335,1.0,0.0,1.0
307,336,1.0,0.0,1.0


In [ ]:
df_caixas_por_loja_highs_s4.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s4

,caixas,10_EN_3,16_EN_3,28_EN_4
0,1,1.0,1.0,0.0
1,2,0.0,0.0,1.0
2,4,1.0,1.0,1.0
3,5,1.0,0.0,1.0
4,6,0.0,0.0,1.0
...,...,...,...,...
304,333,1.0,1.0,1.0
305,334,1.0,0.0,0.0
306,335,1.0,0.0,1.0
307,336,1.0,0.0,1.0


#### Caixas

In [ ]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s4[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s4[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "10_EN_3": 220.0
número de caixas para a chave_loja_buffer_onda "16_EN_3": 119.0
número de caixas para a chave_loja_buffer_onda "28_EN_4": 221.0

total de caixas: 560.0


### 2.2.5- Situação 5

In [ ]:
df_process_situacao5 = df_ffd_s5

df_process_situacao5

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
14,14,1429,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5470920,0.090
15,15,1499,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5470920,0.090
16,16,1499,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3500000,0.400
17,17,909,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3500000,0.400
18,18,1499,2023-10-24,539354347,250,100,55,0.180,1,CARTON_M,...,1375000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1375000,0.180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58412,58412,761,2023-11-21,869448768,702,246,23,0.147,12,CARTON_L,...,3971916,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,47662992,1.764
58413,58413,761,2023-11-21,869448821,295,247,19,0.167,2,CARTON_M,...,1384435,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,2768870,0.334
58414,58414,761,2023-11-21,869448805,407,253,16,0.080,4,CARTON_M,...,1647536,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6590144,0.320
58415,58415,761,2023-11-21,848470202,410,380,35,0.200,1,CARTON_M,...,5453000,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5453000,0.200


#### 2.2.5.1- Entrada de dados no modelo

In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao5['item_loja'] = df_process_situacao5.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao5.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,10_CX_1,2_EN_3,31_EN_3,33_EN_3,3_CX_9
0,1,0.090,0.210,0.210,0.207,0.220
1,2,0.090,0.210,0.140,0.213,0.153
2,3,0.200,0.193,0.455,0.156,0.230
3,4,0.200,0.140,0.313,0.150,0.230
4,5,0.180,0.100,0.400,0.455,0.187
...,...,...,...,...,...,...
399,400,0.027,0.000,0.000,0.000,0.000
400,401,0.147,0.000,0.000,0.000,0.000
401,402,0.137,0.000,0.000,0.000,0.000
402,403,0.143,0.000,0.000,0.000,0.000


In [ ]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao5.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,10_CX_1,2_EN_3,31_EN_3,33_EN_3,3_CX_9
0,1,5470920.0,4773600.0,4773600.0,3228148.0,1556100.0
1,2,5470920.0,4773600.0,1049364.0,2456160.0,1681500.0
2,3,1750000.0,1009120.0,4548648.0,1212300.0,1430100.0
3,4,1750000.0,1049364.0,3324006.0,2290431.0,2275200.0
4,5,1375000.0,1796535.0,6075000.0,4548648.0,2096496.0
...,...,...,...,...,...,...
399,400,670670.0,0.0,0.0,0.0,0.0
400,401,3555576.0,0.0,0.0,0.0,0.0
401,402,3482136.0,0.0,0.0,0.0,0.0
402,403,3961375.0,0.0,0.0,0.0,0.0


In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao5['item_loja'] = df_process_situacao5.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao5.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,10_CX_1,2_EN_3,31_EN_3,33_EN_3,3_CX_9
0,1,1.0,1.0,1.0,1.0,1.0
1,2,1.0,1.0,1.0,1.0,1.0
2,3,1.0,1.0,1.0,1.0,1.0
3,4,1.0,1.0,1.0,1.0,1.0
4,5,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...
399,400,1.0,0.0,0.0,0.0,0.0
400,401,1.0,0.0,0.0,0.0,0.0
401,402,1.0,0.0,0.0,0.0,0.0
402,403,1.0,0.0,0.0,0.0,0.0


In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao5['item_loja'] = df_process_situacao5.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao5.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,10_CX_1,2_EN_3,31_EN_3,33_EN_3,3_CX_9
0,1,1.0,1.0,1.0,1.0,7.0
1,2,1.0,1.0,1.0,1.0,6.0
2,3,2.0,1.0,2.0,1.0,17.0
3,4,2.0,1.0,1.0,2.0,4.0
4,5,1.0,5.0,1.0,1.0,12.0
...,...,...,...,...,...,...
399,400,4.0,0.0,0.0,0.0,0.0
400,401,9.0,0.0,0.0,0.0,0.0
401,402,6.0,0.0,0.0,0.0,0.0
402,403,9.0,0.0,0.0,0.0,0.0


Capacidade da caixa como variável

In [ ]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao5.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,10_CX_1,170.0
1,2_EN_3,512.0
2,31_EN_3,512.0
3,33_EN_3,512.0
4,3_CX_9,170.0


In [ ]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao5.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,10_CX_1,83393894.4
1,2_EN_3,75538008.0
2,31_EN_3,75538008.0
3,33_EN_3,75538008.0
4,3_CX_9,83393894.4


In [ ]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [ ]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [ ]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

#### 2.2.5.2- Solvers

In [ ]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=900"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 900
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [3e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e-16, 3e+01]
  RHS    [1e+00, 9e+01]
Presolving model
Presolve: Infeasible

Solving report
  Status            Infeasible
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  Solution status   -
  Timing            0.33 (total)
                    0.26 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)
Coefficient ranges:
  Matrix [3e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 3e+01]
  RHS    [1e+00, 9e+01]
Presolving model
7691 rows, 656505 cols, 2623995 nonzeros  1s
7691 rows, 656505 cols, 2623995 nonzeros  33s
Objective function is integral with scale 1

Solving MIP model with:
   7691 ro

In [ ]:
e_s5_highs = ampl.getVariable('e').getValues().toPandas()
e_s5_highs = e_s5_highs.reset_index()
e_s5_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s5_highs = e_s5_highs.query('valor_multiplo >0')
e_s5_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
71,1,15,2,1
129,1,26,5,5
243,1,49,4,1
704,1,141,5,1
1030,1,207,1,1
...,...,...,...,...
816422,404,70,3,9
817122,404,210,3,5
817220,404,230,1,1
817307,404,247,3,1


In [ ]:
y_s5_highs = ampl.getVariable('y').getValues().toPandas()
y_s5_highs = y_s5_highs.reset_index()
y_s5_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s5_highs = y_s5_highs.query('selecao>0')

df_caixas_por_loja_highs_s5 = y_s5_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s5 = df_caixas_por_loja_highs_s5.reset_index()
df_caixas_por_loja_highs_s5.columns.name = None
df_caixas_por_loja_highs_s5 # as colunas são as lojas

,Caixa,1,2,3,4,5
0,2,0.0,1.0,1.0,1.0,0.0
1,3,0.0,1.0,1.0,1.0,0.0
2,4,0.0,0.0,1.0,0.0,0.0
3,5,0.0,1.0,0.0,0.0,0.0
4,6,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...
308,400,0.0,0.0,0.0,0.0,1.0
309,402,0.0,0.0,1.0,0.0,0.0
310,403,1.0,0.0,0.0,0.0,0.0
311,404,1.0,0.0,0.0,0.0,1.0


In [ ]:
df_caixas_por_loja_highs_s5.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s5

,caixas,10_CX_1,2_EN_3,31_EN_3,33_EN_3,3_CX_9
0,2,0.0,1.0,1.0,1.0,0.0
1,3,0.0,1.0,1.0,1.0,0.0
2,4,0.0,0.0,1.0,0.0,0.0
3,5,0.0,1.0,0.0,0.0,0.0
4,6,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...
308,400,0.0,0.0,0.0,0.0,1.0
309,402,0.0,0.0,1.0,0.0,0.0
310,403,1.0,0.0,0.0,0.0,0.0
311,404,1.0,0.0,0.0,0.0,1.0


#### Caixas

In [ ]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s5[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s5[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "10_CX_1": 49.0
número de caixas para a chave_loja_buffer_onda "2_EN_3": 139.0
número de caixas para a chave_loja_buffer_onda "31_EN_3": 138.0
número de caixas para a chave_loja_buffer_onda "33_EN_3": 125.0
número de caixas para a chave_loja_buffer_onda "3_CX_9": 50.0

total de caixas: 501.0
